 encoding utf-8 
'''
@File       : .py
@Contact    : username.you@email.address.com
@License    : (C)Copyright AIQ Solutions 2022

@ModifyTime     @Author     @Version        @Description
-----------     -------     --------        ------------
  username    1.0             None

Module Documentation: 

'''

In [7]:
import nrrd
import os
import numpy as np
import glob
from collections import OrderedDict
import pandas as pd
import shutil
import numpy as np
import napari
from skimage import data, filters  # Just to generate some test data (3D blobs).

In [8]:
def myLabelStack(label_list):
    """given a list of str types, read in nrrd files and convert into one image array."""
    img3 = 0
    header = OrderedDict()
    for lab in label_list:
        mask = nrrd.read(lab)
        img3 = img3 + mask[0]

    header = mask[1]
    nrrd_img3 = (img3, header)
    return nrrd_img3

In [9]:
def myNormaliseLabelImg3d(img3):
    """ given an 3D ndarry, normalise the data values and return. """
    img3_normalised = (img3 - np.min(img3))/(np.max(img3) - np.min(img3))
    return img3_normalised

In [10]:
def myNormalisedLabelStack(label_list):
    """ given a list of str types, read in nrrd files, normalise values, and convert into one image array.
    returns: (tuple) image stack containing ndarry data and header information. """
    # mask = nrrd.read(label_list[0])
    # img3 = np.zeros(shape = mask[0].shape)
    img3 = 0
    header = OrderedDict()
    for lab in label_list:
        mask = nrrd.read(lab)
        img3 = img3 + mask[0]

        # print(mask[1])

    header = mask[1] # assume all file headers are the same and can just take the last one.
    overlap_index = np.where(img3>255)
    img3[overlap_index] = 255
    img3_stack = (myNormaliseLabelImg3d(img3), header)
    # img3_stack = (img3, header)

    return img3_stack

In [91]:
# # remove combined nrrd
# root_study_dir = 'D:\\Prostate02_Test_Retest\\Testing'
# scan_dir = os.listdir(root_study_dir)
# scan_dir.sort()
# # i = 0
# for scan in scan_dir:
#     if "." not in scan:
#         print("Now processing:" + str(scan))

#         scan_modal = os.listdir(os.path.join(root_study_dir,scan))
#         scan_modal.sort()
#         for scan_modal_each in scan_modal:
#             if scan_modal_each[0] != "." and os.path.isdir(os.path.join(root_study_dir,scan,scan_modal_each)):
#                 if scan_modal_each == "nrrd":
#                     for files in os.listdir(os.path.join(root_study_dir,scan,scan_modal_each)):
#                         if 'lesionData' in files:
#                             os.remove(os.path.join(root_study_dir,scan,scan_modal_each,files))
#                             print('Removed: ', os.path.join(root_study_dir,scan,scan_modal_each,files))

In [12]:
# for all patients:
# convert individual lesions into lesion stacks
# normalise mask into binary mask
# convert labels of lesions into specific numbers
root_dir = 'D:\\Prostate02_Test_Retest\\Sorted_Repeatability_Test_Retest_Jake'
scans = os.listdir(root_dir)
scans.sort()
# subjects = sorted(glob.glob(root_dir+'PS069*')) #'PS001LB'
# subject_id = [os.path.basename(f) for f in subjects]

for s in scans:
    if '.' not in s:
        print('processing: ', s)
        # output_dir = s + ''
        list_of_all_mask = sorted(glob.glob(os.path.join(root_dir,s,'nrrd')+'\\*mask*.nrrd'))
        lesion_mask = []
        for mask in list_of_all_mask:
            if 'Ref' not in mask and 'Burden' not in mask:
                lesion_mask.append(mask)
                print(mask)
                
        lesion_mask_count = len(lesion_mask)
        print('mask total no.: ', lesion_mask_count)
        
        if lesion_mask_count != 0:
            img3d = myNormalisedLabelStack(lesion_mask)
            filename = os.path.join(root_dir,s,'nrrd',s.split(' ')[0].replace('-', '_') + '_lesionData.nrrd') 
            nrrd.write(filename, img3d[0], img3d[1])  # str, data, header
            print('written combined lesion mask: ', s)
        
        # total_count = 0
        # for file in lesion_mask:
        #     mask = nrrd.read(file)
        #     lesion_index = np.where(mask[0] > 0)
        #     print(len(lesion_index[2]))
        #     total_count += len(lesion_index[2])


processing:  RP-01 1
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 1\nrrd\mask_19-1.nrrd
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 1\nrrd\mask_20-1.nrrd
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 1\nrrd\mask_25-1.nrrd
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 1\nrrd\mask_25-2.nrrd
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 1\nrrd\mask_25-3.nrrd
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 1\nrrd\mask_26-1.nrrd
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 1\nrrd\mask_30-1.nrrd
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 1\nrrd\mask_30-2.nrrd
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 1\nrrd\mask_30-3.nrrd
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_Jake\RP-01 1\nrrd\mask_44-1.nrrd
D:\Prostate02_Test_Retest\Sorted_Repeatability_Test_Retest_

In [87]:
mask = nrrd.read('D:\\Prostate02_Test_Retest\\Testing\\RP-01 1\\nrrd\\RP_01_lesionData.nrrd')
np.max(mask[0])
odd = np.where(mask[0]>255)
mask[0][odd] = 255
# lesion_index = np.where(mask[0] > 0)
# df = pd.DataFrame(lesion_index)

# df.to_csv("D:\\Prostate02_Test_Retest\\new.csv")
# print(lesion_index)

In [88]:
odd = np.where(mask[0]>255)
mask[0][odd]

array([], dtype=float64)

In [65]:
# mask_all = nrrd.read('D:\\Prostate02_Test_Retest\\Sorted_Repeatability_Test_Retest_Jake\\RP-13 1\\nrrd\\RP_13_lesionData.nrrd')
# mask_37_3 = nrrd.read('D:\\Prostate02_Test_Retest\\Sorted_Repeatability_Test_Retest_Jake\\RP-13 1\\nrrd\\mask_37-3.nrrd')
# mask_37_2 = nrrd.read('D:\\Prostate02_Test_Retest\\Sorted_Repeatability_Test_Retest_Jake\\RP-13 1\\nrrd\\mask_37-2.nrrd')


# img3 = 0
# img3 += mask_37_3[0]
# img3 += mask_37_2[0]

# mask_37_2[0].shape

# len(np.where(img3 > 0)[1])
# len(np.where(mask_37_2[0] > 0)[1])